In [1]:
from elasticsearch import Elasticsearch, helpers
import yaml
import json
import bucketstore
import randomname


In [2]:
with open("secrets.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

bucketstore.login(
    access_key_id=cfg['spaces']['access'],
    secret_access_key=cfg['spaces']['secret'],
    region='nyc3',
    endpoint_url=cfg['spaces']['url']
)
bucket = bucketstore.get('wrathalake')

elastic = Elasticsearch(
    cloud_id=cfg['elastic']['cloudId'],
    api_key=(cfg['elastic']['apiKey'])
)

ridesKey = 'intermediate/rides.json'


In [3]:
rides = json.loads(bucket[ridesKey])


In [4]:
for ride in rides.keys():
    if rides[ride]['inElastic'] == False:
        print("Need to add data for ride {} into Elastic".format(ride))
        
        rideData = json.loads(bucket[rides[ride]['interFile']])
        rideName = '{}-{}'.format(rides[ride]['rideDate'], randomname.get_name())
        
        elasticData = [
            {
                '_op_type': 'index',
                '_index': 'equilab-rides',
                '_source': {
                    '@timestamp': point['time'],
                    'rideName': rideName,
                    'location': {
                        'lat': point['coords']['lat'],
                        'lon': point['coords']['long']
                    },
                    'elevation': point['elevation'],
                    'timeDelta': point.get('timeDelta', None),
                    'distance': point.get('distance', None),
                    'speed': point.get('speed', None),
                    'climb': point.get('climb', None),
                    'drop': point.get('drop', None)
                }
            }
            for point in rideData
        ]
        
        try:
            # make the bulk call, and get a response
            response = helpers.bulk(elastic, elasticData)

            #response = helpers.bulk(elastic, actions, index='employees', doc_type='people')
            print ("\nRESPONSE:", response)
        except Exception as e:
            print("\nERROR:", e)

Need to add data for ride raw/sources/equilab/training-2023-09-09.gpx into Elastic

RESPONSE: (6437, [])
Need to add data for ride raw/sources/equilab/training-2023-09-10.gpx into Elastic

RESPONSE: (6912, [])
Need to add data for ride raw/sources/equilab/training-2023-09-23.gpx into Elastic

RESPONSE: (9083, [])
Need to add data for ride raw/sources/equilab/training-2023-10-14.gpx into Elastic

RESPONSE: (14040, [])
Need to add data for ride raw/sources/equilab/training-2023-10-22.gpx into Elastic

RESPONSE: (10471, [])
